In [21]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras

# Install keras_tuner
!pip install -q -U keras-tuner
import keras_tuner as kt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Modifying the feature columns

# drop columns
application_df = application_df.drop(columns=['EIN', 'NAME', 'AFFILIATION', 'SPECIAL_CONSIDERATIONS', 'ORGANIZATION', 'USE_CASE'])
application_df.head()

,APPLICATION_TYPE,CLASSIFICATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,1,0,5000,1
1,T3,C2000,1,1-9999,108590,1
2,T5,C3000,1,0,5000,0
3,T3,C2000,1,10000-24999,6692,1
4,T3,C1000,1,100000-499999,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE      17
CLASSIFICATION        71
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Using 300 instead of 500
application_types_to_replace = application_counts[application_counts < 300].index 

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [10]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Using 300 instead of 1000
classifications_to_replace = list(classification_counts[classification_counts < 300].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummified_df = pd.get_dummies(application_df, dtype = int)
application_dummified_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,CLASSIFICATION_Other,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [17]:
# Split our preprocessed data into our features and target arrays
X = application_dummified_df.drop("IS_SUCCESSFUL", axis=1)
y = application_dummified_df["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1)

In [18]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_inputs = X_train.shape[1]
hidden_nodes_l1 =80
hidden_nodes_l2 = 50
hidden_nodes_l3 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l1, activation = 'relu', input_dim = number_inputs))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l2, activation = 'relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l3, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                2240      
                                                                 
 dense_1 (Dense)             (None, 50)                4050      
                                                                 
 dense_2 (Dense)             (None, 25)                1275      
                                                                 
 dense_3 (Dense)             (None, 1)                 26        
                                                                 
Total params: 7591 (29.65 KB)
Trainable params: 7591 (29.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer= "adam", metrics = ["accuracy"])

In [24]:
# Train the model
# Create a callback that saves the model's weights every five epochs
callback_path = "ASC callbacks"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = callback_path, save_freq = 'epoch', save_weights_only = True, verbose = 1, period = 5)

fit_model = nn.fit(X_train_scaled, y_train, epochs = 100, callbacks = cp_callback)

Epoch 1/100
804/804 [==============================] - 1s 671us/step - loss: 0.6421 - accuracy: 0.6158
Epoch 2/100
804/804 [==============================] - 1s 666us/step - loss: 0.6364 - accuracy: 0.6259
Epoch 3/100
804/804 [==============================] - 1s 653us/step - loss: 0.6348 - accuracy: 0.6247
Epoch 4/100
804/804 [==============================] - 1s 650us/step - loss: 0.6341 - accuracy: 0.6253
Epoch 5/100
773/804 [===========================>..] - ETA: 0s - loss: 0.6339 - accuracy: 0.6233
Epoch 5: saving model to ASC callbacks
804/804 [==============================] - 1s 683us/step - loss: 0.6344 - accuracy: 0.6231
Epoch 6/100
804/804 [==============================] - 1s 653us/step - loss: 0.6336 - accuracy: 0.6234
Epoch 7/100
804/804 [==============================] - 1s 655us/step - loss: 0.6335 - accuracy: 0.6264
Epoch 8/100
804/804 [==============================] - 1s 662us/step - loss: 0.6337 - accuracy: 0.6241
Epoch 9/100
804/804 [==============================]

804/804 [==============================] - 1s 652us/step - loss: 0.6306 - accuracy: 0.6248
Epoch 65/100
781/804 [============================>.] - ETA: 0s - loss: 0.6305 - accuracy: 0.6271
Epoch 65: saving model to ASC callbacks
804/804 [==============================] - 1s 656us/step - loss: 0.6304 - accuracy: 0.6264
Epoch 66/100
804/804 [==============================] - 1s 682us/step - loss: 0.6305 - accuracy: 0.6270
Epoch 67/100
804/804 [==============================] - 1s 657us/step - loss: 0.6303 - accuracy: 0.6274
Epoch 68/100
804/804 [==============================] - 1s 658us/step - loss: 0.6303 - accuracy: 0.6251
Epoch 69/100
804/804 [==============================] - 1s 651us/step - loss: 0.6303 - accuracy: 0.6253
Epoch 70/100
750/804 [==========================>...] - ETA: 0s - loss: 0.6301 - accuracy: 0.6264
Epoch 70: saving model to ASC callbacks
804/804 [==============================] - 1s 682us/step - loss: 0.6303 - accuracy: 0.6262
Epoch 71/100
804/804 [=============

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6411 - accuracy: 0.6282 - 165ms/epoch - 617us/step
Loss: 0.641115665435791, Accuracy: 0.6282215714454651


## Second Optimization

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_inputs = X_train.shape[1]
hidden_nodes_l1 =80
hidden_nodes_l2 = 60
hidden_nodes_l3 = 30
hidden_nodes_l4 = 10

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l1, activation = 'relu', input_dim = number_inputs))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l2, activation = 'relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l3, activation = 'relu'))

#Fourth hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l4, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 80)                2240      
                                                                 
 dense_5 (Dense)             (None, 60)                4860      
                                                                 
 dense_6 (Dense)             (None, 30)                1830      
                                                                 
 dense_7 (Dense)             (None, 10)                310       
                                                                 
 dense_8 (Dense)             (None, 1)                 11        
                                                                 
Total params: 9251 (36.14 KB)
Trainable params: 9251 (36.14 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [27]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer= "adam", metrics = ["accuracy"])

In [28]:
# Train the model
# Create a callback that saves the model's weights every five epochs
callback_path = "ASC callbacks"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = callback_path, save_freq = 'epoch', save_weights_only = True, verbose = 1, period = 5)

fit_model = nn.fit(X_train_scaled, y_train, epochs = 100, callbacks = cp_callback)

Epoch 1/100
804/804 [==============================] - 1s 737us/step - loss: 0.6408 - accuracy: 0.6194
Epoch 2/100
804/804 [==============================] - 1s 729us/step - loss: 0.6351 - accuracy: 0.6238
Epoch 3/100
804/804 [==============================] - 1s 715us/step - loss: 0.6347 - accuracy: 0.6238
Epoch 4/100
804/804 [==============================] - 1s 715us/step - loss: 0.6340 - accuracy: 0.6238
Epoch 5/100
781/804 [============================>.] - ETA: 0s - loss: 0.6343 - accuracy: 0.6247
Epoch 5: saving model to ASC callbacks
804/804 [==============================] - 1s 732us/step - loss: 0.6340 - accuracy: 0.6247
Epoch 6/100
804/804 [==============================] - 1s 723us/step - loss: 0.6336 - accuracy: 0.6260
Epoch 7/100
804/804 [==============================] - 1s 720us/step - loss: 0.6328 - accuracy: 0.6249
Epoch 8/100
804/804 [==============================] - 1s 713us/step - loss: 0.6330 - accuracy: 0.6239
Epoch 9/100
804/804 [==============================]

804/804 [==============================] - 1s 713us/step - loss: 0.6306 - accuracy: 0.6276
Epoch 65/100
771/804 [===========================>..] - ETA: 0s - loss: 0.6307 - accuracy: 0.6267
Epoch 65: saving model to ASC callbacks
804/804 [==============================] - 1s 732us/step - loss: 0.6305 - accuracy: 0.6266
Epoch 66/100
804/804 [==============================] - 1s 723us/step - loss: 0.6304 - accuracy: 0.6274
Epoch 67/100
804/804 [==============================] - 1s 720us/step - loss: 0.6300 - accuracy: 0.6259
Epoch 68/100
804/804 [==============================] - 1s 707us/step - loss: 0.6304 - accuracy: 0.6253
Epoch 69/100
804/804 [==============================] - 1s 716us/step - loss: 0.6301 - accuracy: 0.6261
Epoch 70/100
777/804 [===========================>..] - ETA: 0s - loss: 0.6302 - accuracy: 0.6269
Epoch 70: saving model to ASC callbacks
804/804 [==============================] - 1s 727us/step - loss: 0.6305 - accuracy: 0.6263
Epoch 71/100
804/804 [=============

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6398 - accuracy: 0.6294 - 161ms/epoch - 599us/step
Loss: 0.63984215259552, Accuracy: 0.6293877363204956


## Third Optimization

In [76]:
#  Import and read the charity_data.csv.
import pandas as pd 
application_df2 = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df2.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [77]:
# Modifying the feature columns

# drop columns
application_df2 = application_df2.drop(columns=['EIN', 'NAME', 'AFFILIATION', 'SPECIAL_CONSIDERATIONS'])
application_df2.head()

,APPLICATION_TYPE,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,C1000,ProductDev,Association,1,0,5000,1
1,T3,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,C3000,ProductDev,Association,1,0,5000,0
3,T3,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,C1000,Heathcare,Trust,1,100000-499999,142590,1


In [78]:
# Determine the number of unique values in each column.
application_df2.nunique()

APPLICATION_TYPE      17
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [79]:
# Look at APPLICATION_TYPE value counts for binning
application_counts2 = application_df2["APPLICATION_TYPE"].value_counts()
application_counts2

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [80]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Back to 500 from trying 300 which yielded a lower percentage 

application_types_to_replace = list(application_counts2[application_counts2 < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df2['APPLICATION_TYPE'] = application_df2['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df2['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [81]:
# Look at CLASSIFICATION value counts for binning
classification_count2 = application_df2["CLASSIFICATION"].value_counts()
classification_count2

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [82]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count2[classification_count2 > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [83]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
# Back to using 1000 after trying 300 which yielded a lower percentage

classifications_to_replace = list(classification_count2[classification_count2 < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df2['CLASSIFICATION'] = application_df2['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df2['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [91]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummified_df2 = pd.get_dummies(application_df2, dtype = int)
application_dummified_df2.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [92]:
# Split our preprocessed data into our features and target arrays
X = application_dummified_df2.drop("IS_SUCCESSFUL", axis = 1)
y = application_dummified_df2["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [93]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [94]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Changing activation this time around

number_inputs = X_train.shape[1]
hidden_nodes_l1 =80
hidden_nodes_l2 = 60
hidden_nodes_l3 = 40
hidden_nodes_l4 = 25
hidden_nodes_l5 = 10 

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l1, activation = 'sigmoid', input_dim = number_inputs))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l2, activation = 'sigmoid'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l3, activation = 'sigmoid'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l4, activation = 'sigmoid'))

# Fifth hidden layer 
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l5, activation = 'sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 80)                2880      
                                                                 
 dense_15 (Dense)            (None, 60)                4860      
                                                                 
 dense_16 (Dense)            (None, 40)                2440      
                                                                 
 dense_17 (Dense)            (None, 25)                1025      
                                                                 
 dense_18 (Dense)            (None, 10)                260       
                                                                 
 dense_19 (Dense)            (None, 1)                 11        
                                                                 
Total params: 11476 (44.83 KB)
Trainable params: 11476

In [95]:
# Compile the model
nn.compile(loss = "binary_crossentropy", optimizer= "adam", metrics = ["accuracy"])

In [96]:
# Train the model
# Create a callback that saves the model's weights every five epochs
callback_path = "ASC callbacks"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = callback_path, save_freq = 'epoch', save_weights_only = True, verbose = 1, period = 5)

fit_model = nn.fit(X_train_scaled, y_train, epochs = 100, callbacks = cp_callback)

Epoch 1/100
804/804 [==============================] - 1s 773us/step - loss: 0.6854 - accuracy: 0.5672
Epoch 2/100
804/804 [==============================] - 1s 775us/step - loss: 0.6326 - accuracy: 0.6509
Epoch 3/100
804/804 [==============================] - 1s 784us/step - loss: 0.6296 - accuracy: 0.6527
Epoch 4/100
804/804 [==============================] - 1s 775us/step - loss: 0.6293 - accuracy: 0.6537
Epoch 5/100
791/804 [============================>.] - ETA: 0s - loss: 0.6277 - accuracy: 0.6530
Epoch 5: saving model to ASC callbacks
804/804 [==============================] - 1s 782us/step - loss: 0.6276 - accuracy: 0.6532
Epoch 6/100
804/804 [==============================] - 1s 787us/step - loss: 0.6252 - accuracy: 0.6543
Epoch 7/100
804/804 [==============================] - 1s 769us/step - loss: 0.6211 - accuracy: 0.6526
Epoch 8/100
804/804 [==============================] - 1s 773us/step - loss: 0.6183 - accuracy: 0.6530
Epoch 9/100
804/804 [==============================]

804/804 [==============================] - 1s 756us/step - loss: 0.6021 - accuracy: 0.6597
Epoch 65/100
795/804 [============================>.] - ETA: 0s - loss: 0.6027 - accuracy: 0.6595
Epoch 65: saving model to ASC callbacks
804/804 [==============================] - 1s 775us/step - loss: 0.6017 - accuracy: 0.6606
Epoch 66/100
804/804 [==============================] - 1s 759us/step - loss: 0.6018 - accuracy: 0.6579
Epoch 67/100
804/804 [==============================] - 1s 754us/step - loss: 0.6014 - accuracy: 0.6602
Epoch 68/100
804/804 [==============================] - 1s 759us/step - loss: 0.6016 - accuracy: 0.6600
Epoch 69/100
804/804 [==============================] - 1s 760us/step - loss: 0.6012 - accuracy: 0.6616
Epoch 70/100
796/804 [============================>.] - ETA: 0s - loss: 0.6014 - accuracy: 0.6595
Epoch 70: saving model to ASC callbacks
804/804 [==============================] - 1s 773us/step - loss: 0.6014 - accuracy: 0.6599
Epoch 71/100
804/804 [=============

In [97]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6126 - accuracy: 0.6578 - 142ms/epoch - 530us/step
Loss: 0.6126236319541931, Accuracy: 0.6578425765037537


## Optimization with Hyperparameter options

In [98]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras_tuner import RandomSearch, Hyperband
from keras.callbacks import ModelCheckpoint

In [99]:
#  Import and read the charity_data.csv.
import pandas as pd
application_df3 = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df3.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [100]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df3 = application_df3.drop(["EIN", "NAME"], axis = 1)

In [101]:
# Determine the number of unique values in each column.
application_df3.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [102]:
# Look at APPLICATION_TYPE value counts for binning
application_counts3 = application_df3["APPLICATION_TYPE"].value_counts()
application_counts3

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [103]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts3[application_counts3 < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df3['APPLICATION_TYPE'] = application_df3['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df3['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [104]:
# Look at CLASSIFICATION value counts for binning
classification_count3 = application_df3["CLASSIFICATION"].value_counts()
classification_count3

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [105]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count3[classification_count3 > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [106]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_count3[classification_count3 < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df3['CLASSIFICATION'] = application_df3['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df3['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [107]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummified_df3 = pd.get_dummies(application_df3, dtype = int)
application_dummified_df3.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [108]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [109]:
def create_model(hp):
    model = Sequential()
    activation = hp.Choice("activation", ["relu","tanh","sigmoid"])
    
    model.add(Dense(units = hp.Int("num_neurons", 1,10, step=3),
                   activation = activation,
                    input_dim = X_train.shape[1]
                   ))
    
    for x in range(hp.Int("num_layers", 1,6)):
        model.add(Dense(units = hp.Int("num_layers_" + str(x+2), 1 , 10, step = 2),
                       activation=activation
                       ))
        
    model.add(Dense(units=1, activation = "sigmoid"))
    
    model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    return model

In [110]:
tuner = Hyperband(
                    create_model,
                    objective = 'val_accuracy',
                    max_epochs=10,
                    hyperband_iterations=2,
                    overwrite = True
)

In [111]:
# Use Keras Tuner to find best hyperparameters
tuner.search(X_train_scaled, y_train, epochs = 5, validation_data = (X_test_scaled, y_test))

Trial 60 Complete [00h 00m 06s]
val_accuracy: 0.5292128324508667

Best val_accuracy So Far: 0.6598250865936279
Total elapsed time: 00h 02m 48s


In [112]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'num_neurons': 10,
 'num_layers': 2,
 'num_layers_2': 5,
 'num_layers_3': 3,
 'num_layers_4': 3,
 'num_layers_5': 9,
 'num_layers_6': 3,
 'num_layers_7': 1,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0020'}

In [113]:
# Evaluate best model against full test data

best_model = tuner.get_best_models(1)[0]

model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)

print(f" loss : {model_loss}, accuracy : {model_accuracy}")

268/268 - 0s - loss: 0.6196 - accuracy: 0.6598 - 162ms/epoch - 604us/step
 loss : 0.6195617914199829, accuracy : 0.6598250865936279


In [114]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Changing activation this time around

number_inputs = X_train.shape[1]
hidden_nodes_l1 =80
hidden_nodes_l2 = 60
hidden_nodes_l3 = 40
hidden_nodes_l4 = 30
hidden_nodes_l5 = 10 

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l1, activation = 'relu', input_dim = number_inputs))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l2, activation = 'relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l3, activation = 'relu'))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l4, activation = 'relu'))

# Fifth hidden layer 
nn.add(tf.keras.layers.Dense(units = hidden_nodes_l5, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'relu'))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 80)                2880      
                                                                 
 dense_5 (Dense)             (None, 60)                4860      
                                                                 
 dense_6 (Dense)             (None, 40)                2440      
                                                                 
 dense_7 (Dense)             (None, 30)                1230      
                                                                 
 dense_8 (Dense)             (None, 10)                310       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 11731 (45.82 KB)
Trainable params: 11731

In [116]:
# Compile 
nn.compile(loss = "binary_crossentropy", optimizer= "adam", metrics = ["accuracy"])

In [117]:
# Train the model
# Create a callback that saves the model's weights every five epochs
callback_path = "ASC callbacks"
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = callback_path, save_freq = 'epoch', save_weights_only = True, verbose = 1, period = 5)

fit_model = nn.fit(X_train_scaled, y_train, epochs = 100, callbacks = cp_callback)

Epoch 1/100
804/804 [==============================] - 1s 755us/step - loss: 0.7288 - accuracy: 0.6245
Epoch 2/100
804/804 [==============================] - 1s 762us/step - loss: 0.6544 - accuracy: 0.6429
Epoch 3/100
804/804 [==============================] - 1s 747us/step - loss: 0.6384 - accuracy: 0.6443
Epoch 4/100
804/804 [==============================] - 1s 748us/step - loss: 0.6410 - accuracy: 0.6525
Epoch 5/100
749/804 [==========================>...] - ETA: 0s - loss: 0.6270 - accuracy: 0.6524
Epoch 5: saving model to ASC callbacks
804/804 [==============================] - 1s 770us/step - loss: 0.6294 - accuracy: 0.6482
Epoch 6/100
804/804 [==============================] - 1s 763us/step - loss: 0.6412 - accuracy: 0.6523
Epoch 7/100
804/804 [==============================] - 1s 767us/step - loss: 0.6341 - accuracy: 0.6444
Epoch 8/100
804/804 [==============================] - 1s 754us/step - loss: 0.6310 - accuracy: 0.6569
Epoch 9/100
804/804 [==============================]

804/804 [==============================] - 1s 742us/step - loss: 0.6121 - accuracy: 0.6610
Epoch 65/100
752/804 [===========================>..] - ETA: 0s - loss: 0.6172 - accuracy: 0.6607
Epoch 65: saving model to ASC callbacks
804/804 [==============================] - 1s 753us/step - loss: 0.6159 - accuracy: 0.6610
Epoch 66/100
804/804 [==============================] - 1s 740us/step - loss: 0.6188 - accuracy: 0.6590
Epoch 67/100
804/804 [==============================] - 1s 735us/step - loss: 0.6154 - accuracy: 0.6596
Epoch 68/100
804/804 [==============================] - 1s 742us/step - loss: 0.6168 - accuracy: 0.6612
Epoch 69/100
804/804 [==============================] - 1s 743us/step - loss: 0.6141 - accuracy: 0.6595
Epoch 70/100
785/804 [============================>.] - ETA: 0s - loss: 0.6083 - accuracy: 0.6603
Epoch 70: saving model to ASC callbacks
804/804 [==============================] - 1s 785us/step - loss: 0.6085 - accuracy: 0.6604
Epoch 71/100
804/804 [=============

In [118]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6857 - accuracy: 0.6588 - 143ms/epoch - 535us/step
Loss: 0.6857223510742188, Accuracy: 0.6587755084037781


In [119]:
# Export our model to HDF5 file
nn.save(filepath = 'AlphabetSoupCharity_Optimization.h5')

/Users/lazy_boy/anaconda3/envs/ml/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
